<a href="https://colab.research.google.com/github/pratyushraman10/Customer-and-Sales-Behavior-Analysis-of-Online-Retail-Transactions/blob/main/Deployment_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [22]:
# 📦 Step 1: Import Required Libraries
import os
import json
import nltk

In [23]:
# 🧹 Step 2: Clean Previous NLTK Downloads (Optional)
import shutil
shutil.rmtree('/root/nltk_data', ignore_errors=True)


In [24]:
# 📥 Step 3: Download Required NLTK Tokenizer
nltk.download('punkt_tab')  # This includes the sentence tokenizer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [25]:
# 🔁 Step 4: Rename .crdownload Files to .txt (if needed)
for filename in os.listdir():
    if filename.endswith(".crdownload"):
        new_filename = filename.replace(".crdownload", "")
        os.rename(filename, new_filename)
        print(f"✅ Renamed: {filename} → {new_filename}")

In [26]:
# 📁 Step 5: Define Text Files and Author Styles
text_files = {
    "eugene_onegin.txt": "__style__pushkin",
    "war_and_peace.txt": "__style__tolstoy",
    "anna_karenina.txt": "__style__tolstoy",
    "idiot.txt": "__style__dostoevsky",
    "karamzov.txt": "__style__dostoevsky",
    "crime_and_punishment.txt": "__style__dostoevsky",
    "white_nights.txt": "__style__dostoevsky",
    "chekov_1.txt": "__style__chekhov",
    "chekov_2.txt": "__style__chekhov"
}

In [27]:
# 📊 Step 6: Collect All Style-Tagged Sentences
dataset = []

for filename, style in text_files.items():
    if not os.path.exists(filename):
        print(f"⚠️ File not found: {filename}")
        continue

    try:
        with open(filename, "r", encoding="utf-8") as f:
            text = f.read()

        # Normalize text
        text = text.replace("\n", " ").replace("\r", " ").strip()

        # Sentence tokenization
        sentences = nltk.sent_tokenize(text)

        # Filter and collect
        for sentence in sentences:
            sentence = sentence.strip()
            if 40 < len(sentence) < 300:
                dataset.append({
                    "text": sentence,
                    "style": style
                })

        print(f"✅ Processed {filename} — {len(sentences)} sentences")

    except Exception as e:
        print(f"❌ Error in {filename}: {e}")


✅ Processed eugene_onegin.txt — 2028 sentences
✅ Processed war_and_peace.txt — 26551 sentences
✅ Processed anna_karenina.txt — 16908 sentences
✅ Processed idiot.txt — 12854 sentences
✅ Processed karamzov.txt — 19837 sentences
✅ Processed crime_and_punishment.txt — 12060 sentences
✅ Processed white_nights.txt — 6979 sentences
✅ Processed chekov_1.txt — 4420 sentences
✅ Processed chekov_2.txt — 75783 sentences


In [28]:
# 💾 Step 7: Save to JSONL
with open("russian_literature_dataset.jsonl", "w", encoding="utf-8") as outfile:
    for item in dataset:
        outfile.write(json.dumps(item, ensure_ascii=False) + "\n")

In [29]:
print("🎉 All done! Saved to 'russian_literature_dataset.jsonl'")

🎉 All done! Saved to 'russian_literature_dataset.jsonl'


In [30]:
!pip install transformers datasets accelerate peft bitsandbytes --quiet
!pip install -U huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.5 MB/s eta 0:00:00


In [31]:
from google.colab import files
uploaded = files.upload()


Saving russian_literature_dataset.jsonl to russian_literature_dataset (1).jsonl


In [32]:
from datasets import load_dataset, Dataset
import pandas as pd

In [38]:
# Load JSONL
df = pd.read_json("russian_literature_dataset.jsonl", lines=True)
# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [39]:
# Prepend style token to text
def add_style_prefix(example):
    example["input_text"] = f"{example['style']} {example['text']}"
    return example

dataset = dataset.map(add_style_prefix)

Map:   0%|          | 0/114464 [00:00<?, ? examples/s]

In [40]:
# Train/validation split
dataset = dataset.train_test_split(test_size=0.1)

In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

model_checkpoint = "gpt2"  # Or use "gpt2-medium" or "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 has no pad token

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [42]:
# Tokenize inputs
def tokenize(example):
    return tokenizer(example["input_text"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/103017 [00:00<?, ? examples/s]

Map:   0%|          | 0/11447 [00:00<?, ? examples/s]

In [43]:
# Data collator for masked language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [46]:
# Training config
training_args = TrainingArguments(
    output_dir="./gpt2-russian-style",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",  # Changed from evaluation_strategy
    save_strategy="epoch",
    logging_steps=20,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False
)

In [47]:
# Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipython-input-47-3051070003.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ramanpratyush101 (ramanpratyush101-nit-rourkela) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,2.553800,2.659903
2,2.451000,2.615883
3,2.391800,2.610085


TrainOutput(global_step=77265, training_loss=2.5301147576220204, metrics={'train_runtime': 9356.5516, 'train_samples_per_second': 33.03, 'train_steps_per_second': 8.258, 'total_flos': 2.0188140945408e+16, 'train_loss': 2.5301147576220204, 'epoch': 3.0})

In [48]:
def generate_text(prompt, style="__style__dostoevsky"):
    input_ids = tokenizer(f"{style} {prompt}", return_tensors="pt").input_ids.to(model.device)
    output = model.generate(input_ids, max_new_tokens=100, temperature=0.7, do_sample=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [49]:
# Try it
print(generate_text("A man walks into a dark alley,", style="__style__tolstoy"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


__style__tolstoy A man walks into a dark alley, and by the light of the lantern sees a peasant.”  “A peasant!” Natásha answered, and was silent. “Why, he’s a man of the greatest nobility, and has had a fortune of a million.”  “Oh, no, he’s not rich, that’s true, but at least a little.”  “What is that?” asked the doctor.  “


In [50]:
!pip install gradio --quiet


In [51]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


In [54]:
# Load your fine-tuned model and tokenizer
# model_path = "./gpt2-russian-style" # No need to load again, objects are in memory after training

# Reuse the tokenizer and model objects from the previous training step
# tokenizer = AutoTokenizer.from_pretrained(model_path) # Redundant loading
# model = AutoModelForCausalLM.from_pretrained(model_path) # Redundant loading

model.to("cuda" if torch.cuda.is_available() else "cpu")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [55]:
# Generation function
def generate_text(prompt, author_style, max_tokens=100, temperature=0.7):
    input_text = f"{author_style} {prompt}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_tokens,
        temperature=temperature,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text


In [56]:
# Gradio interface
interface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="e.g., A man walked into the snow..."),
        gr.Dropdown(choices=[
            "__style__dostoevsky",
            "__style__tolstoy",
            "__style__pushkin",
            "__style__chekhov"
        ], label="Author Style"),
        gr.Slider(50, 250, value=100, step=10, label="Max Tokens"),
        gr.Slider(0.1, 1.0, value=0.7, step=0.1, label="Temperature")
    ],
    outputs="text",
    title="🪶 Russian Author Style Text Generator",
    description="Generate literary passages in the style of Russian greats. Fine-tuned GPT-2."
)

interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f059a601b0ac38441a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [57]:
interface.launch(share=True)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f059a601b0ac38441a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
